In [ ]:
### CNN in Tensorflow ###

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.models import Model


In [ ]:
mnist_data = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist_data .load_data()

training_images, test_images = training_images/255.0, test_images/255.0

print(training_images.shape)

training_images = np.expand_dims(training_images, -1)
test_images = np.expand_dims(test_images, -1)
print(training_images.shape)

output_classes = len(set(training_labels))
print("Number of output classes is: ", output_classes)

training_images[0].shape

# Assuming you have defined 'training_images' and 'output_classes' appropriately
input_layer = Input(shape=training_images[0].shape)
conv1 = Conv2D(32, (3,3), strides=2, activation='relu')(input_layer)
maxpool1 = MaxPooling2D((2, 2))(conv1)
conv2 = Conv2D(64, (3,3), strides=2, activation='relu')(maxpool1)
# If you need another convolutional layer
# conv3 = Conv2D(128, (3,3), strides=2, activation='relu')(conv2)
flat1 = Flatten()(conv2)
drop1 = Dropout(0.2)(flat1)
dense1 = Dense(512, activation='relu')(drop1)
drop2 = Dropout(0.2)(dense1)
output_layer = Dense(output_classes, activation='softmax')(drop2)
model = Model(input_layer, output_layer)

# Compile and fit the model as necessary
model.compile(optimizer = 'adam', loss= 'sparse_categorical_crossentropy', metrics =['accuracy'])

model_history = model.fit(training_images, training_labels,
epochs=20, validation_data=(test_images, test_labels),
verbose=1)

output = model.predict(test_images)
prediction = np.argmax(output[9])
print(prediction)

In [ ]:
### CNN in Tensorflow for CIFAR-10 dataset###

In [25]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.models import Model


# Load CIFAR-10 dataset
cifar_dataset = tf.keras.datasets.cifar10
(training_images, training_labels), (test_images, test_labels) = cifar_dataset.load_data()

# Normalize pixel values to be between 0 and 1
training_images, test_images = training_images / 255.0, test_images / 255.0

print("Training images shape:", training_images.shape)

# No need to expand dimensions as CIFAR-10 images are already in 3 channels
print("Training images shape after expanding dimensions:", training_images.shape)

output_classes = len(np.unique(training_labels))
print("Number of output classes is:", output_classes)

# Model architecture
input_layer = Input(shape=training_images.shape[1:])
conv1 = Conv2D(32, (3,3), strides=2, activation='relu')(input_layer)
maxpool1 = MaxPooling2D((2, 2))(conv1)
conv2 = Conv2D(64, (3,3), strides=2, activation='relu')(maxpool1)
flat1 = Flatten()(conv2)
drop1 = Dropout(0.2)(flat1)
dense1 = Dense(512, activation='relu')(drop1)
drop2 = Dropout(0.2)(dense1)
output_layer = Dense(output_classes, activation='softmax')(drop2)

model = Model(input_layer, output_layer)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_history = model.fit(training_images, training_labels, epochs=5, validation_data=(test_images, test_labels), verbose=1)

# Predict using the model
output = model.predict(test_images)
prediction = np.argmax(output[9])
print("Prediction for the 10th test image:", prediction)


Training images shape: (50000, 32, 32, 3)
Training images shape after expanding dimensions: (50000, 32, 32, 3)
Number of output classes is: 10
Epoch 1/5
1563/1563 [==============================] - 7s 4ms/step - loss: 1.5579 - accuracy: 0.4350 - val_loss: 1.3122 - val_accuracy: 0.5186
Epoch 2/5
1563/1563 [==============================] - 6s 4ms/step - loss: 1.2610 - accuracy: 0.5509 - val_loss: 1.1753 - val_accuracy: 0.5769
Epoch 3/5
1563/1563 [==============================] - 6s 4ms/step - loss: 1.1369 - accuracy: 0.5974 - val_loss: 1.0491 - val_accuracy: 0.6222
Epoch 4/5
1563/1563 [==============================] - 8s 5ms/step - loss: 1.0426 - accuracy: 0.6327 - val_loss: 0.9722 - val_accuracy: 0.6598
Epoch 5/5
313/313 [==============================] - 1s 2ms/step
Prediction for the 10th test image: 1


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(test_images[9])
plt.colorbar()
plt.grid(False)
plt.show()

In [28]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


class MyCIFARCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        # Adjust the accuracy threshold as needed
        if logs.get('accuracy') > 0.998:
            print("\nReached 99.8% accuracy so cancelling training!")
            self.model.stop_training = True


# Load and preprocess CIFAR-10 dataset
(training_images, training_labels), (test_images, test_labels) = cifar10.load_data()
training_images, test_images = training_images / 255.0, test_images / 255.0

# Flatten the training labels for unique class count
unique_training_labels = tf.unique(tf.reshape(training_labels, [-1]))[0]

# Model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=training_images.shape[1:]),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.2),
    Dense(512, activation='relu'),
    Dropout(0.2),
    # Output layer with number of classes dynamically determined
    Dense(len(unique_training_labels), activation='softmax')
])


# Compile the model
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Set up callbacks
callbacks = MyCIFARCallback()

# Train the model
model_history = model.fit(training_images, training_labels, 
                          epochs=5, 
                          validation_data=(test_images, test_labels), 
                          callbacks=[callbacks])

# Predict using the model
predictions = model.predict(test_images)
prediction = tf.argmax(predictions[9]).numpy()

print("Prediction for the 10th test image:", prediction)


Epoch 1/5
1563/1563 [==============================] - 16s 10ms/step - loss: 1.3994 - accuracy: 0.4966 - val_loss: 1.1535 - val_accuracy: 0.5880
Epoch 2/5
1563/1563 [==============================] - 15s 10ms/step - loss: 1.0619 - accuracy: 0.6267 - val_loss: 0.9982 - val_accuracy: 0.6464
Epoch 3/5
1563/1563 [==============================] - 16s 10ms/step - loss: 0.9103 - accuracy: 0.6806 - val_loss: 0.8853 - val_accuracy: 0.6969
Epoch 4/5
1563/1563 [==============================] - 16s 10ms/step - loss: 0.7958 - accuracy: 0.7229 - val_loss: 0.8217 - val_accuracy: 0.7178
Epoch 5/5
313/313 [==============================] - 2s 5ms/step
Prediction for the 10th test image: 1


In [ ]:
### LeNet(CNN) ###

In [ ]:
from keras.datasets import mnist
from keras.utils import to_categorical  # Updated import

# Load dataset as train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Set numeric type to float32 from uint8
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

# Normalize value to [0, 1]
x_train /= 255
x_test /= 255

# Transform labels to one-hot encoding
y_train = to_categorical(y_train, 10)  # Updated function call
y_test = to_categorical(y_test, 10)    # Updated function call

# Reshape the dataset into 4D array
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)


from keras.models import Sequential
from keras import layers

# Instantiate an empty model
model = Sequential()

# C1 convolutional layer
model.add(layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=(28, 28, 1), padding="same"))

# S2 pooling layer
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))

# C3 convolutional layer
model.add(layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))

# S4 pooling layer
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

# C5 fully connected convolutional layer
model.add(layers.Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))

# Flatten the CNN output so that we can connect it with fully connected layers
model.add(layers.Flatten())

# FC6 fully connected layer
model.add(layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])

# Train the model by calling model.fit() function and pass in the training data, number of epochs, and batch size.
# Keras provides a facility to evaluate the loss and accuracy at the end of each epoch
hist = model.fit(x=x_train, y=y_train, epochs=5, batch_size=128, validation_data=(x_test, y_test), verbose=1)

# Evaluate the model by calling model.evaluate
test_score = model.evaluate(x_test, y_test)
print("Test loss: {:.4f}, accuracy: {:.2f}%".format(test_score[0], test_score[1]*100))


In [ ]:
### LeNet(CNN) adapted to CIFAR-10 dataset ###

In [37]:
from keras.datasets import cifar10

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert class vectors to binary class matrices (one-hot encoding)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


# Instantiate an empty model
model = Sequential()

# Adapted C1 convolutional layer for CIFAR-10
model.add(layers.Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=(32, 32, 3), padding='same'))

# S2 pooling layer
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

# Adapted C3 convolutional layer for CIFAR-10
model.add(layers.Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))

# S4 pooling layer
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

# C5 fully connected convolutional layer
model.add(layers.Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))

# Flatten the CNN output so that we can connect it with fully connected layers
model.add(layers.Flatten())

# FC6 fully connected layer
model.add(layers.Dense(84, activation='tanh'))  # Increased to 84 neurons as per the original LeNet-5 paper

# Output layer
model.add(layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

# Train the model
hist = model.fit(x_train, y_train, epochs=5, batch_size=128, validation_data=(x_test, y_test), verbose=1)

# Evaluate the model
test_score = model.evaluate(x_test, y_test)
print("Test loss: {:.4f}, accuracy: {:.2f}%".format(test_score[0], test_score[1]*100))

Epoch 1/5
391/391 [==============================] - 8s 21ms/step - loss: 2.0814 - accuracy: 0.2433 - val_loss: 1.9541 - val_accuracy: 0.2955
Epoch 2/5
391/391 [==============================] - 8s 20ms/step - loss: 1.8910 - accuracy: 0.3299 - val_loss: 1.8311 - val_accuracy: 0.3547
Epoch 3/5
391/391 [==============================] - 8s 21ms/step - loss: 1.8031 - accuracy: 0.3637 - val_loss: 1.7685 - val_accuracy: 0.3777
Epoch 4/5
391/391 [==============================] - 9s 22ms/step - loss: 1.7488 - accuracy: 0.3839 - val_loss: 1.7916 - val_accuracy: 0.3731
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 1.6896 - accuracy: 0.4028
Test loss: 1.6896, accuracy: 40.28%


In [ ]:
### Optimized LeNet Code for CIFAR-10 ###

In [38]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Define the custom callback for early stopping
class MyCIFARCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') > 0.998:
            print("\nReached 99.8% accuracy so cancelling training!")
            self.model.stop_training = True

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert class vectors to binary class matrices (one-hot encoding)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Instantiate the LeNet-5 model with updated architecture for CIFAR-10
model = Sequential([
    Conv2D(32, (5, 5), activation='tanh', input_shape=(32, 32, 3), padding='same'),
    AveragePooling2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(32, (5, 5), activation='tanh', padding='valid'),
    AveragePooling2D(pool_size=(2, 2), strides=(2, 2)),
    Flatten(),
    Dense(84, activation='tanh'),
    Dense(10, activation='softmax')
])

# Compile the model with 'adam' optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Set up callbacks for early stopping
callbacks = MyCIFARCallback()

# Train the model
model_history = model.fit(x_train, y_train, 
                          epochs=5, 
                          batch_size=128, 
                          validation_data=(x_test, y_test), 
                          callbacks=[callbacks],
                          verbose=1)

# Evaluate the model
test_score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss: {:.4f}, accuracy: {:.2f}%".format(test_score[0], test_score[1]*100))

# Predict using the model
predictions = model.predict(x_test)
prediction = tf.argmax(predictions[9]).numpy()
print("Prediction for the 10th test image:", prediction)


Epoch 1/5
391/391 [==============================] - 8s 20ms/step - loss: 1.7987 - accuracy: 0.3637 - val_loss: 1.6673 - val_accuracy: 0.4143
Epoch 2/5
391/391 [==============================] - 7s 19ms/step - loss: 1.5901 - accuracy: 0.4424 - val_loss: 1.5066 - val_accuracy: 0.4725
Epoch 3/5
391/391 [==============================] - 10s 25ms/step - loss: 1.4427 - accuracy: 0.4932 - val_loss: 1.3989 - val_accuracy: 0.5034
Epoch 4/5
391/391 [==============================] - 10s 25ms/step - loss: 1.3416 - accuracy: 0.5242 - val_loss: 1.3493 - val_accuracy: 0.5164
Epoch 5/5
391/391 [==============================] - 10s 25ms/step - loss: 1.2755 - accuracy: 0.5464 - val_loss: 1.3161 - val_accuracy: 0.5368
Test loss: 1.3161, accuracy: 53.68%
313/313 [==============================] - 1s 4ms/step
Prediction for the 10th test image: 1
